# Imports

In [66]:
from sqlalchemy import create_engine
import pandas as pd

import sqlite3

# Load data and EDA

In [67]:
df_equip = pd.read_csv('data/raw/russia_losses_equipment.csv')
df_equip.head()

,date,day,aircraft,helicopter,tank,APC,field artillery,MRL,military auto,fuel tank,drone,naval ship,anti-aircraft warfare,special equipment,mobile SRBM system,greatest losses direction,vehicles and fuel tanks,cruise missiles
0,2022-02-25,2,10,7,80,516,49,4,100.0,60.0,0,2,0,NaN,NaN,NaN,NaN,NaN
1,2022-02-26,3,27,26,146,706,49,4,130.0,60.0,2,2,0,NaN,NaN,NaN,NaN,NaN
2,2022-02-27,4,27,26,150,706,50,4,130.0,60.0,2,2,0,NaN,NaN,NaN,NaN,NaN
3,2022-02-28,5,29,29,150,816,74,21,291.0,60.0,3,2,5,NaN,NaN,NaN,NaN,NaN
4,2022-03-01,6,29,29,198,846,77,24,305.0,60.0,3,2,7,NaN,NaN,NaN,NaN,NaN


In [68]:
df_equip.describe()

,day,aircraft,helicopter,tank,APC,field artillery,MRL,military auto,fuel tank,drone,naval ship,anti-aircraft warfare,special equipment,mobile SRBM system,vehicles and fuel tanks,cruise missiles
count,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,65.000000,65.000000,339.000000,339.000000,339.000000,320.00000,36.000000,274.000000,274.000000
mean,171.000000,221.870206,199.486726,1886.784661,4098.861357,1113.191740,272.179941,1047.507692,69.323077,877.905605,13.153392,136.091445,101.83125,3.944444,3500.116788,321.751825
std,98.005102,63.459447,57.896681,879.707972,1572.681832,639.702266,117.360921,466.162060,7.545917,586.232462,4.160260,62.177456,56.91926,0.333333,931.763542,216.520039
min,2.000000,10.000000,7.000000,80.000000,516.000000,49.000000,4.000000,100.000000,60.000000,0.000000,2.000000,0.000000,10.00000,2.000000,1796.000000,84.000000
25%,86.500000,204.000000,168.500000,1270.500000,3103.000000,596.000000,200.500000,600.000000,60.000000,461.000000,13.000000,93.000000,51.75000,4.000000,2685.500000,155.000000
50%,171.000000,233.000000,193.000000,1856.000000,4115.000000,978.000000,261.000000,1178.000000,73.000000,779.000000,15.000000,136.000000,99.00000,4.000000,3511.500000,233.000000
75%,255.500000,277.000000,260.000000,2761.500000,5606.000000,1778.500000,391.000000,1437.000000,76.000000,1463.500000,16.000000,202.000000,159.00000,4.000000,4389.250000,480.000000
max,340.000000,293.000000,284.000000,3197.000000,6366.000000,2195.000000,453.000000,1701.000000,76.000000,1947.000000,18.000000,221.000000,199.00000,4.000000,5037.000000,796.000000


In [69]:
df_equip.dtypes

date                          object
day                            int64
aircraft                       int64
helicopter                     int64
tank                           int64
APC                            int64
field artillery                int64
MRL                            int64
military auto                float64
fuel tank                    float64
drone                          int64
naval ship                     int64
anti-aircraft warfare          int64
special equipment            float64
mobile SRBM system           float64
greatest losses direction     object
vehicles and fuel tanks      float64
cruise missiles              float64
dtype: object

In [70]:
df_pers = pd.read_csv('data/raw/russia_losses_personnel.csv')
df_pers.head()

,date,day,personnel,personnel*,POW
0,2022-02-25,2,2800,about,0.0
1,2022-02-26,3,4300,about,0.0
2,2022-02-27,4,4500,about,0.0
3,2022-02-28,5,5300,about,0.0
4,2022-03-01,6,5710,about,200.0


In [71]:
df_pers.describe()

,day,personnel,POW
count,339.000000,339.000000,62.000000
mean,171.000000,52738.660767,386.387097
std,98.005102,31758.235314,131.440363
min,2.000000,2800.000000,0.000000
25%,86.500000,28775.000000,389.000000
50%,171.000000,43400.000000,421.000000
75%,255.500000,75685.000000,474.500000
max,340.000000,126160.000000,496.000000


# Create DB

In [72]:
DB_NAME = "russian_losses.db"

con = sqlite3.connect(DB_NAME)

sql = con.cursor()

# Create tables

## Table 1

In [73]:
list(df_equip)

['date',
 'day',
 'aircraft',
 'helicopter',
 'tank',
 'APC',
 'field artillery',
 'MRL',
 'military auto',
 'fuel tank',
 'drone',
 'naval ship',
 'anti-aircraft warfare',
 'special equipment',
 'mobile SRBM system',
 'greatest losses direction',
 'vehicles and fuel tanks',
 'cruise missiles']

In [74]:
sql.execute('''
    create table if not exists tb_equipment
    (
        observation_date TEXT
        , day_of_war integer
        , aircraft integer null
        , helicopter integer null
        , tank integer null
        , APC integer null
        , field_artilery integer null
        , mrl integer null
        , military_auto integer null
        , fuel_tank integer null
        , drone integer null
        , naval_ship integer null
        , anti_aircraft_warfare integer null
        , special_equipment integer null
        , mobile_SRBM_system integer null
        , greatest_losses_direction TEXT null
        , vehicles_and_fuel_tanks integer null
        , cruise_missiles integer null
    )
''')

In [75]:
sql.execute('''
    select count(1)
    from tb_equipment
''')

print(sql.fetchall())

[(0,)]


## Table 2

In [76]:
list(df_pers)

['date', 'day', 'personnel', 'personnel*', 'POW']

In [77]:
sql.execute('''
    create table if not exists tb_personnel
    (
        observation_date TEXT
        , day_of_war INTEGER
        , personnel ITEGER
        , personnel_about TEXT
        , pows INTEGER
    )
''')

In [78]:
sql.execute('''

''')

In [79]:
sql.execute('''
    select count(1)
    from tb_personnel
''')

print(sql.fetchall())

[(0,)]
